# PB Math Notebook

This notebook aims to compare and contrast how SiRIUS and CASA generate and sample primary beams (PB).
Results are considered identical if they agree for 7 significant digits (32-bit machine precision). The notebook will first discuss Airy disk beams and then holography-based beams.

The C++ code in CASA will be referenced reorganized
- casa6/casa5/code/casatools/synthesis/ CASA < 6.4
- casa6/casatools/src/code/synthesis/   CASA >= 6.4

In [4]:
%config InlineBackend.figure_format = 'svg'

## Unobschured Airy Disk

The objective of this section is to find the PB value for a given FK5 (J2000 in CASA) right ascension ($\alpha$) and declination ($\delta$).
The wikipedia [article](https://en.wikipedia.org/wiki/Airy_disk) has a adequete description of the airy disk. 

$$
I(\theta) = I_0 \frac{2 J_1(x)}{x} = I_0 \frac{2 J_1(k a \sin \theta)}{k a \sin(\theta)}
$$
where <br>
$I$ <br>
$I_0$ <br>
$J_1$ <br>
$x = k a \sin \theta$ <br>
$k$ <br>
$a$ <br>
$\theta$ <br>

\begin{align}
I_0 & sds\\
\end{align}



![im10](https://raw.githubusercontent.com/casangi/sirius/master/docs/_media/Circular_aperture_variables.svg)


### Finding pixels

Don't change anything except phase_center_skycoord

At 1 GHz the approximate full width at half power is 45 arcminutes. NB add plot of airy disk showing it spans 45 arcmites. generate airy disk as function of frequency with slider.

We will be placing our sources at the center of pixels so that we can easily sample values from the pb that gets generated.


In [6]:
import pandas as pd
from IPython.display import display, HTML
from astropy.coordinates import SkyCoord
from sirius._sirius_utils._direction_rotate import _calc_rotation_mats, _sin_project, _tan_project, _local_spherical_coords
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import numpy as np
from astropy import units as u
import math
deg_to_rad = np.pi/180

# Change this and see what happens
phase_center_skycoord = SkyCoord(ra='19h59m28.5s',dec='+40d44m01.5s',frame='fk5')
phase_center = np.array([phase_center_skycoord.ra.rad,phase_center_skycoord.dec.rad])
print(phase_center)


########### Don't change except if really know what you are doing
w = WCS(naxis=2)
image_size = np.array([400,400])
w.wcs.crpix = image_size//2
arcsec_to_deg = 1/3600
w.wcs.cdelt = np.array([5.0,5.0])*arcsec_to_deg
w.wcs.crval = np.array([phase_center_skycoord.ra.degree,phase_center_skycoord.dec.degree])
w.wcs.ctype = ['RA---SIN','DEC--SIN']
#lm_pix_pos = w.all_world2pix(np.array([point_source_skycoord.ra.deg,point_source_skycoord.dec.deg])[None,:], 1)

point_sources_ra_dec_degrees = w.all_pix2world(np.array([[250,250]]),1)
point_source_skycoord = SkyCoord(ra=point_sources_ra_dec_degrees[:,0]*u.deg,dec=point_sources_ra_dec_degrees[:,1]*u.deg,frame='fk5')
print(point_source_skycoord)
point_sources = point_sources_ra_dec_degrees*deg_to_rad
print(point_sources)


pix_pos = w.all_world2pix(point_sources_ra_dec_degrees, 1)
print(pix_pos)
print(point_source_skycoord.ra.hms,point_source_skycoord.dec.dms)


print('%.15f' % point_source_skycoord.ra.rad,'%.15f' % point_source_skycoord.dec.rad)

'''
import pandas as pd
from IPython.display import display, HTML
from astropy.coordinates import SkyCoord
from sirius._sirius_utils._direction_rotate import _calc_rotation_mats, _sin_project, _tan_project, _local_spherical_coords
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import numpy as np
from astropy import units as u
import math
deg_to_rad = np.pi/180

# Change this and see what happens
phase_center_skycoord = SkyCoord(ra='19h59m28.5s',dec='+40d44m00.0s',frame='fk5')
phase_center = np.array([phase_center_skycoord.ra.rad,phase_center_skycoord.dec.rad])
print(phase_center)


########### Don't change except if really know what you are doing
w = WCS(naxis=2)
image_size = np.array([2000,2000])
w.wcs.crpix = image_size//2
arcsec_to_deg = 1/3600
w.wcs.cdelt = np.array([7.2,7.2])*arcsec_to_deg
w.wcs.crval = np.array([phase_center_skycoord.ra.degree,phase_center_skycoord.dec.degree])
w.wcs.ctype = ['RA---SIN','DEC--SIN']
#lm_pix_pos = w.all_world2pix(np.array([point_source_skycoord.ra.deg,point_source_skycoord.dec.deg])[None,:], 1)

point_sources_ra_dec_degrees = w.all_pix2world(np.array([[1000,1000],[900,950],[1400,1400]]),1)
point_source_skycoord = SkyCoord(ra=point_sources_ra_dec_degrees[:,0]*u.deg,dec=point_sources_ra_dec_degrees[:,1]*u.deg,frame='fk5')
print(ps_skycoord)
point_sources = point_sources_ra_dec_degrees*deg_to_rad
print(point_sources)


pix_pos = w.all_world2pix(ps_ra_dec_degrees, 1)
print(pix_pos)
'''

#lm_pix_pos = w.all_world2pix(np.array([point_source_skycoord.ra.deg,point_source_skycoord.dec.deg])[None,:], 1)

'''
#The default setup for the airy disk code is the VLA at 1 GHz
r_max = 0.8564*np.pi/180 #in radians
n = 10000 #sampling
delta_xy = r_max/(n-1) #in radians

print(delta_xy,delta_xy*rad_to_arcsec)
print(delta_xy*rad_to_arcsec*20)

print('delta',delta_xy*rad_to_arcsec*20*400)

phase_center_skycoord = SkyCoord(ra='19h59m28.5s',dec='+40d44m00.0s',frame='fk5')
point_source_skycoord = SkyCoord(ra='19h59m28.5s',dec='+41d32m0.09358668192362529s',frame='fk5') 
print(phase_center_skycoord.dec + (delta_xy*20*200)*u.rad)
print(phase_center_skycoord.dec + (6.1667*200*u.arcsec))

image_center = np.array([2000,2000])//2
cell_size = np.array([-delta_xy,delta_xy])
    
w = WCS(naxis=2)
w.wcs.crpix = image_center
#w.wcs.cdelt = np.array([6.1667,6.1667])*arcsec_to_deg
w.wcs.cdelt = np.array([7.2,7.2])*arcsec_to_deg
w.wcs.crval = np.array([phase_center_skycoord.ra.degree,phase_center_skycoord.dec.degree])
w.wcs.ctype = ['RA---SIN','DEC--SIN']
lm_pix_pos = w.all_world2pix(np.array([point_source_skycoord.ra.deg,point_source_skycoord.dec.deg])[None,:], 1)
ra_dec = w.all_pix2world(np.array([1000,1400])[None,:],1)[0]
print(ra_dec)
point_source_skycoord = SkyCoord(ra=ra_dec[0]*u.deg,dec=ra_dec[1]*u.deg,frame='fk5') 
print(point_source_skycoord.ra.hms,point_source_skycoord.dec.dms)

print(lm_pix_pos)
astro_source_lm_pos = lm_pix_pos*cell_size - image_center*cell_size #tangential plane lm
print(astro_source_lm_pos)
'''

[5.23369701 0.71093805]
<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    [(299.96049139, 40.80315818)]>
[[5.2352982  0.71214946]]
[[250. 250.]]
hms_tuple(h=array([19.]), m=array([59.]), s=array([50.51793355])) dms_tuple(d=array([40.]), m=array([48.]), s=array([11.3694551]))
5.235298200651986 0.712149455487489


"\n#The default setup for the airy disk code is the VLA at 1 GHz\nr_max = 0.8564*np.pi/180 #in radians\nn = 10000 #sampling\ndelta_xy = r_max/(n-1) #in radians\n\nprint(delta_xy,delta_xy*rad_to_arcsec)\nprint(delta_xy*rad_to_arcsec*20)\n\nprint('delta',delta_xy*rad_to_arcsec*20*400)\n\nphase_center_skycoord = SkyCoord(ra='19h59m28.5s',dec='+40d44m00.0s',frame='fk5')\npoint_source_skycoord = SkyCoord(ra='19h59m28.5s',dec='+41d32m0.09358668192362529s',frame='fk5') \nprint(phase_center_skycoord.dec + (delta_xy*20*200)*u.rad)\nprint(phase_center_skycoord.dec + (6.1667*200*u.arcsec))\n\nimage_center = np.array([2000,2000])//2\ncell_size = np.array([-delta_xy,delta_xy])\n    \nw = WCS(naxis=2)\nw.wcs.crpix = image_center\n#w.wcs.cdelt = np.array([6.1667,6.1667])*arcsec_to_deg\nw.wcs.cdelt = np.array([7.2,7.2])*arcsec_to_deg\nw.wcs.crval = np.array([phase_center_skycoord.ra.degree,phase_center_skycoord.dec.degree])\nw.wcs.ctype = ['RA---SIN','DEC--SIN']\nlm_pix_pos = w.all_world2pix(np.array(

In [ ]:
from sirius_data._constants import c
import numpy as np
from astropy.coordinates import SkyCoord
from sirius._sirius_utils._direction_rotate import _calc_rotation_mats, _sin_project, _tan_project, _local_spherical_coords
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from scipy.special import j1
from astropy import units as u
rad_to_arcmin = (60*180)/np.pi
rad_to_arcsec = (3600*180)/np.pi
rad_to_deg = 180/np.pi
arcsec_to_deg = 1/3600

#phase_center_skycoord = SkyCoord(ra='19h59m28.5s',dec='+40d44m00.0s',frame='fk5')
#phase_center = np.array([phase_center_skycoord.ra.rad,phase_center_skycoord.dec.rad])

#point_source_skycoord = SkyCoord(ra='19h59m0.0s',dec='+40d51m01.5s',frame='fk5') 
#point_source_skycoord = SkyCoord(ra='19h59m28.5s',dec='+41d03m33.34734943447074s',frame='fk5') 
#point_source_skycoord = SkyCoord(ra='19h59m28.5s',dec='+41d25m6.738798313801908s',frame='fk5') 
#point_source_skycoord = SkyCoord(ra='19h59m28.5s',dec='+41d32m0.09358668192362529s',frame='fk5') 
#point_source = np.array([point_source_skycoord.ra.rad,point_source_skycoord.dec.rad])





### The CASA Implmentation of the Airy Disk 

- The Airy disk is sampled for 10000 points at 1 GHz for a given radius. Truncation (int) is used instead of rounding.
- Truncated constants are used.

Simulator code uses apply(SkyComponent& in, SkyComponent& out, ...) in TransformMachines/PBMath1D.cc

- Small angle approximation is used

The small


In [ ]:
# The twiddle factor is needed to improve agreement between CASA PBMATH airy disk and SiRIUS. 
# This arrises because CASA makes use of truncated constants. The twiddle factor 0.9998277835716939 is very close to 1. 
# Even with the twiddle there will not be machine precission agreement, because CASA calculates a 1D PB at 1 GHz for 10000 points and then frequency scales and interpolates to desired value.
casa_twiddle = (180*7.016*c)/((np.pi**2)*(10**9)*1.566*24.5) # 0.9998277835716939
r_vla_max = 0.8564*np.pi/180 #in radians

print('r_vla_max',r_vla_max)

print((10000-1)/(0.8564*60))

print((10000-1)/((r_vla_max*60*180)/np.pi))

def airy_disk(r,twiddle=casa_twiddle,freq=10**9,dish_diameter=24.5,r_max=r_vla_max,n_sample=10000,use_round=False,p=2):
    with np.errstate(divide='ignore',invalid='ignore'):
        k = (2*np.pi*freq)/c
        a = dish_diameter/2

        if n_sample is not None:
            r_inc = ((r_max)/(n_sample-1))
            #print('Index',int(r/r_inc))
            #r = (int(r/r_inc)*r_inc)*a*k
            if use_round:
                
                r = ((np.floor(r/r_inc + 0.5)).astype(int)*r_inc)*a*k
                print('2 r/r_inc',(r-(np.floor(r/r_inc + 0.5))*r_inc)/r_inc)
            else:
                r = ((r/r_inc).astype(int)*r_inc)*a*k
                print('2 r/r_inc',(r-(r/r_inc).astype(int)*r_inc)/r_inc)
        else:
            r = r*a*k

        pb_val = (2.0*j1(r*twiddle)/(r*twiddle))**p
        pb_val[np.isnan(pb_val)] = 1

        return pb_val


def calc_theta(ra_dec_o,ra_dec):
    #Equation 6 http://tdc-www.harvard.edu/wcstools/aips27.pdf
    ra = ra_dec[0]
    dec = ra_dec[1]
    ra_o = ra_dec_o[0]
    dec_o = ra_dec_o[1]
    theta = np.arccos(np.sin(dec)*np.sin(dec_o) + np.cos(dec)*np.cos(dec_o)*np.cos(ra-ra_o))
    return theta





### CASA 





In [ ]:
#CASA method


#casa6/browse/casatools/src/code/synthesis/TransformMachines/BeamSkyJones.cc BeamSkyJones::apply(SkyComponent& in,..)
#casa6/browse/casatools/src/code/synthesis/TransformMachines/PBMath1D.cc PBMath1D::apply(SkyComponent& in, ...)
#casa6/browse/casatools/src/code/synthesis/TransformMachines/PBMath1DAiry PBMath1DAiry::fillPBArray


# The _calc_rotation_mats is machine precssion identical to the code in https://github.com/casacore/casacore/blob/dbf28794ef446bbf4e6150653dbe404379a3c429/measures/Measures/UVWMachine.cc.
# The only difference is that the sign flip of the uv coordinates are no longer required (the ms and casacore of different definiations of the uvw coordinates).
# lmn_rot is a point on the celestial sphere measured from the intersection of the celestial sphere and the phase_center (the l,m=0 point).

# CASA
lmn_rot = np.zeros((len(point_sources),3))
for i in range(len(point_sources)): _, lmn_rot[i,:] = _calc_rotation_mats(phase_center, point_sources[i,:])
sep = 2.0*np.arcsin(np.sqrt(np.sum(lmn_rot**2,axis=1))/2.0)  #np.sin(), small angle approximation
pb = airy_disk(sep,freq=3*10**9)
pb_results_0 = np.append(sep, pb)
print(pb_results_0)

# No Small andle approx
sep = np.sin(2.0*np.arcsin(np.sqrt(np.sum(lmn_rot**2,axis=1))/2.0))  #np.sin(), small angle approximation
pb = airy_disk(sep,freq=3*10**9)
pb_results_1 = np.append(sep, pb)

# No Small Angle approx and No twiddle
sep = np.sin(2.0*np.arcsin(np.sqrt(np.sum(lmn_rot**2,axis=1))/2.0))  #np.sin(), small angle approximation
pb = airy_disk(sep,twiddle=1,freq=3*10**9)
pb_results_2 = np.append(sep, pb)

# Sirius: No Small Angle approx, No twiddle and no sampleing
sep = np.sin(2.0*np.arcsin(np.sqrt(np.sum(lmn_rot**2,axis=1))/2.0))  #np.sin(), small angle approximation
pb = airy_disk(sep,twiddle=1,n_sample=None,freq=3*10**9)
pb_results_3 = np.append(sep, pb)




pb_results = np.vstack((pb_results_0,pb_results_1,pb_results_2,pb_results_3))
print(pb_results)

#results = {'r ps_0':pb_results[0,:],

#pb_results_df = pd.DataFrame(data=pb_results, index = ['CASA','No SAA', 'No SAA, No twiddle','SiRIUS (No SAA, No twiddle, No Sample)'] , columns=['r ps_0','r ps_1',' r ps_2','pb ps_0','pb ps_1', 'pb ps_2'])
pb_results_df = pd.DataFrame(data=pb_results, index = ['CASA','No SAA', 'No SAA, No twiddle','SiRIUS (No SAA, No twiddle, No Sample)'] , columns=['r ps_0','pb ps_0'])



with pd.option_context('display.float_format', '{:0.7f}'.format):
    display(HTML(pb_results_df.to_html()))

print(pb_results_df.to_numpy(),pb_results_df.to_numpy().shape)
    
per_dif = (pb_results_df.to_numpy()[:3,:]-pb_results_df.to_numpy()[1:,:])/pb_results_df.to_numpy()[:3,:]
per_dif = 100*np.vstack((per_dif,(pb_results_df.to_numpy()[0] - pb_results_df.to_numpy()[3])/pb_results_df.to_numpy()[0]))
#per_dif_df = pd.DataFrame(data=per_dif, index = ['CASA vs No SAA','No SAA vs (No SAA, No twiddle)', '(No SAA, No twiddle) vs (SiRIUS (No SAA, No twiddle, No Sample))','CASA vs SiRIUS'] , columns=['r ps_0','r ps_1',' r ps_2','pb ps_0','pb ps_1', 'pb ps_2'])
per_dif_df = pd.DataFrame(data=per_dif, index = ['CASA vs No SAA','No SAA vs (No SAA, No twiddle)', '(No SAA, No twiddle) vs (SiRIUS (No SAA, No twiddle, No Sample))','CASA vs SiRIUS'] , columns=['r ps_0','pb ps_0'])


print(per_dif)

with pd.option_context('display.float_format', '{:0.7f}'.format):
    display(HTML(per_dif_df.to_html()))

    

#_, lmn_rot= map(_calc_rotation_mats, phase_center, point_sources)

'''
_, lmn_rot = _calc_rotation_mats(phase_center, point_sources) 
print('lmn_rot',lmn_rot)

sep = 2.0*np.arcsin(np.sqrt(np.sum(lmn_rot**2))/2.0)  #np.sin(), small angle approximation
print(sep)
#print(sep)
#sep=0.005688552

pb = airy_disk(sep)
print('pb',pb)
'''

In [ ]:
#The standard gridder creation of the PB does not make use of the of the small angle approximation. 

sep = np.sin(2.0*np.arcsin(np.sqrt(np.sum(lmn_rot**2,axis=1))/2.0))  #np.sin(), small angle approximation
print(sep)
#print(sep)
#sep=0.005688552

pb = airy_disk(sep)

pb_results_1 = np.append(sep, pb)

print(pb_results_1)
print('pb',pb)

In [ ]:
(0.008169312478301662-0.008169221611796491)
import pandas as pd
from IPython.display import display, HTML

pb_results = np.vstack((pb_results_0,pb_results_1))
print(pb_results)

#results = {'r ps_0':pb_results[0,:],

pb_results_df = pd.DataFrame(data=pb_results, index = ['CASA','CASA No Small Ang Approx'] , columns=['r ps_0','r ps_1',' r ps_2','pb ps_0','pb ps_1', 'pb ps_2'])

with pd.option_context('display.float_format', '{:0.7f}'.format):
    display(HTML(pb_results_df.to_html()))


### SiRIUS

\begin{align}
l &= \cos \delta \sin \Delta \alpha \\
m &= \sin \delta \cos \delta_0 - \cos \delta \sin \delta_0 \cos \Delta \alpha
\end{align}

$\Delta \alpha \equiv \alpha - \alpha_0$

In [ ]:
# SiRIUS Method (maybe?)
# The _sin_project function does orthographic/syntehsis projection of a right ascension and declination skycoord (point_source_skycoord) to a tangential plane with center at phase_center.
# The code is machine precssion identical to the wcs library.
# The equations in sections 3.2.3 and 3.2.4 of http://tdc-www.harvard.edu/wcstools/aips27.pdf are used.

lm_sin = np.zeros((len(point_sources),2))
for i in range(len(point_sources)): lm_sin[i,:] = _sin_project(phase_center, point_sources[i,:])


print(lm_sin)
sep = np.sqrt(lm_sin[:,0]**2 + lm_sin[:,1]**2)
print(sep)

pb = airy_disk(sep,twiddle=1,n_sample=None)
print('pb',pb)

pb_results_1 = np.append(sep, pb)



'''
_, lmn_rot = _calc_rotation_mats(phase_center, point_sources) 
sep = np.sin(2.0*np.arcsin(np.sqrt(np.sum(lmn_rot**2))/2.0))  
print(sep)

#add wcs calculation to prove same 


pb = airy_disk(sep,twiddle=1,n_sample=None)
print('pb',pb)
'''

In [ ]:
%load_ext autoreload
%autoreload 2

\begin{align}
\theta &= \arccos(\sin\delta \sin \delta_0 + \cos \delta \cos \delta_0  \cos \Delta \alpha) \\
\phi &= \arctan2(\cos \delta \sin \Delta \alpha, \sin \delta \cos \delta_0 + \cos \delta \cos \delta_0 \cos \Delta \alpha)
\end{align}



In [ ]:
#theta = calc_theta(phase_center, point_source)
theta_phi = _local_spherical_coords(phase_center, point_source)
print(theta_phi)
sep = np.sin(theta_phi[0])
print(sep)

pb = airy_disk(sep)
print('pb',pb)

In [ ]:
lm_tan = _tan_project(phase_center, point_source)
print('lm_tan',lm_tan)

sep = np.sqrt(lm_tan[0]**2 + lm_tan[1]**2)
print(sep)
pb = airy_disk(sep*k*a, twiddle)
print('pb',pb)

In [ ]:
from simple_sim import single_field_source_sim
single_field_source_sim()

In [ ]:
from casatasks import tclean
import os
os.system('rm -rf /.lustre/cv/users/jsteeb/sirius_demo/try0*')
tclean(vis='/.lustre/cv/users/jsteeb/sirius_demo/sim_data.ms', 
       imagename='/.lustre/cv/users/jsteeb/sirius_demo/try0',
       datacolumn='data',
       imsize=[2000,2000],
       cell='6.1667arcsec',
       specmode='mfs',
       gridder='standard',
       niter=1000,
       pbmask=0.0,
       pblimit=0.0,
       gain=0.3)

#Also Run the mosaic and awproject gridder to show difference in PB (maybe?). 

In [ ]:
#The default setup for the airy disk code is the VLA at 1 GHz
r_max = 0.8564*np.pi/180 #in radians
n = 10000 #sampling
delta_xy = r_max/(n-1) #in radians

print(delta_xy,delta_xy*rad_to_arcsec)
print(delta_xy*rad_to_arcsec*20)

print('delta',delta_xy*rad_to_arcsec*20*400)

phase_center_skycoord = SkyCoord(ra='19h59m28.5s',dec='+40d44m00.0s',frame='fk5')
point_source_skycoord = SkyCoord(ra='19h59m28.5s',dec='+41d32m0.09358668192362529s',frame='fk5') 
print(phase_center_skycoord.dec + (delta_xy*20*200)*u.rad)
print(phase_center_skycoord.dec + (6.1667*200*u.arcsec))

image_center = np.array([2000,2000])//2
cell_size = np.array([-delta_xy,delta_xy])
    
w = WCS(naxis=2)
w.wcs.crpix = image_center
#w.wcs.cdelt = np.array([6.1667,6.1667])*arcsec_to_deg
w.wcs.cdelt = np.array([7.2,7.2])*arcsec_to_deg
w.wcs.crval = np.array([phase_center_skycoord.ra.degree,phase_center_skycoord.dec.degree])
w.wcs.ctype = ['RA---SIN','DEC--SIN']
lm_pix_pos = w.all_world2pix(np.array([point_source_skycoord.ra.deg,point_source_skycoord.dec.deg])[None,:], 1)
ra_dec = w.all_pix2world(np.array([1000,1400])[None,:],1)[0]
print(ra_dec)
point_source_skycoord = SkyCoord(ra=ra_dec[0]*u.deg,dec=ra_dec[1]*u.deg,frame='fk5') 
print(point_source_skycoord.ra.hms,point_source_skycoord.dec.dms)

print(lm_pix_pos)
astro_source_lm_pos = lm_pix_pos*cell_size - image_center*cell_size #tangential plane lm
print(astro_source_lm_pos)